In [ ]:
source(paste0(dirname(dirname(dirname(getwd()))),'/map.r'))
source(paste0(HELP_DIR, "shortcuts.r"))
source(paste0(HELP_DIR, "helpers.r"))
source(paste0(HELP_DIR, "fisher.r"))

In [ ]:
library(survminer)
library(rlang)
library(cowplot)
library(patchwork)
library(ggpubr)

# 0 - Read in base data

In [ ]:
go <- 
fread(paste0(TMP_DIR, "pfs_highlights.csv")) %>% 
 se(example, cohortGo, feature, clean_feature) %>% 
 mu(type = ifelse( grepl("and", feature), "combination", "univariate")) %>% 
 se(cohortGo, feature, type, clean_feature)

In [ ]:
combos <- readRDS(paste0(TMP_DIR, "combo_features.Rds"))

In [ ]:
ready <- 
fread(paste0(SHARE_DIR, "fisher_base.csv")) %>% 
 se( sampleId, cohortGo, daysToPfsEvent, pfsEvent, any_of( go %>% pu(feature )))  %>% 
 lj(combos[['Pan-Cancer / Immunotherapy']] %>% se(sampleId, purity_tmbPerMb_lt8_and_rna_geneset_KEGG_HEDGEHOG_SIGNALING_PATHWAY_gt75), by = "sampleId") %>% 
 lj(combos[['Pan-Cancer / Pembrolizumab']] %>% se(sampleId, rna_geneset_gene_set_t_cell_gep_6_lt75_and_rna_geneset_KEGG_TGF_BETA_SIGNALING_PATHWAY_gt50), by = "sampleId")

In [ ]:
names(ready) <- c("sampleId", "cohortGo", "daysToPfsEvent", "pfsEvent", gsub("-", "_", gsub("/","_",gsub(" ", "_", go %>% pu(clean_feature)))))

In [ ]:
#head(ready)

# 1 - Survival Images

In [ ]:
data_prep <- function(x, cohort) {
    
    data <- ready %>% fi( cohortGo == !! cohort) %>% drop_na( !!sym(x) )
    
    surv_formula <- expr(Surv(daysToPfsEvent, pfsEvent) ~ !!sym(x))
    fits <- survfit(eval(surv_formula, envir = environment()), data = data)
    pval <- signif(survdiff(eval(surv_formula, envir = environment()), data = data)$pvalue,2)

    list("data" = data, "fits" = fits, "surv_formula" = surv_formula, "pval" = pval)
}

In [ ]:
figurer <- function(x, cohort, data, fits, pval, pad = 30 ) {
        
    # Compute max time in data
    max_time <- max(data$daysToPfsEvent, na.rm = TRUE)
    xmax <- min(max_time, 1100)
    
    #fit
    oo <- 
    ggsurvplot(
     fits,
     data = data,
     palette = c("#7AABD3", "#e52f28"),
     conf.int = TRUE, 
     risk.table = TRUE, 
     pval.coord = c(700, .95), 
     xlim = c(0, xmax), 
     break.time.by = 300, 
     ggtheme = theme_minimal(),
     xlab = "Days", 
     ylab = "Survival Probability", 
     title = cohort) 

    oo$plot <- 
    oo$plot + 
     annotate("text", x = 500, y = 0.9, label = paste0("Log-rank        \np-value = ", pval), size = 5) + 
     theme(plot.title = element_text(hjust = 0.5)) + 
     guides(color = guide_legend(nrow = 2, byrow = TRUE)) 

    as_ggplot(gridExtra::arrangeGrob(oo$plot, oo$table, layout_matrix = matrix(c(1,1,1,1,1,1,1,2,2)))) + 
    theme(plot.margin = margin(t = pad, r = pad, b = pad, l = pad))
}

In [ ]:
survival_figures <- list()
for( i in seq(nrow(go))){
    cohortGo <- as.character(go[i,1])
    feature <- gsub("-", "_", gsub("/", "_", gsub(" ", "_", as.character(go[i,4]))))

    data_ready <- data_prep( feature, cohortGo )

    data <- data_ready$data 
    fits <- data_ready$fits
    pval <- data_ready$pval
    surv_formula <- data_ready$surv_formula

    survival_figures[[i]] <- figurer( feature, cohortGo, data = data, fits = fits, pval = pval)
}

In [ ]:
length(survival_figures)

In [ ]:
options(repr.plot.height = 19, repr.plot.width = 26) 
wrap_plots(survival_figures[1:12], ncol = 4)

In [ ]:
#options(repr.plot.height = 20, repr.plot.width = 17) 
ggsave(paste0(SHARE_DIR, "5_PFS_paper.png"), wrap_plots(survival_figures[1:12], ncol = 4), width = 26, height = 19)